From basic GET, pull:
* name
* DBLINKS
* formula
* reaction
* pathway
* module
* enzyme
* Brite?

In [8]:
import numpy as np
import pandas as pd
import requests
import time
import re

def grab_subfield(contents, query_key):
    """Grabs content of a raw subfield from listified raw text response from GET. 
    """
    subfield_bool = 0
    subfield_contents_ls = []
    for i in range(len(contents)):
        ln = contents[i]
        # Switch on reader
        if ln[:len(query_key)] == query_key:
            subfield_bool = 1
            switch_idx = i
            subfield_contents_ls.append(ln)
        if ln[:12] == "            " and subfield_bool == 1:
            subfield_contents_ls.append(ln)
        # switch off reader
        if subfield_bool == 1 and ln[:12] != "            " and i > switch_idx:
            subfield_bool = 0
    
    subfield_contents_ls = [x[12:] for x in subfield_contents_ls]
    
    return subfield_contents_ls


def parse_compound_info(contents, flat_object=True):
    """cpd:C01290
    
    PARAMS
    ------
    contents: "\n"-split string from kegg GET request.
    flat_object: bool. If True, returns a dictionary of lists. further levels will be "|"-delimited. 
    """
    full_dict = {}

    # ===== NAME =====
    names_ls = grab_subfield(contents, "NAME")
    # drop the semi-colon from the end of each name if it's there
    for i in range(len(names_ls)):
        if names_ls[i][-1] == ";":
            names_ls[i] = names_ls[i][:-1]
    full_dict["name"] = grab_subfield(contents, "NAME")

    # ===== DBLINKS =====
    subfield_content = grab_subfield(contents, "DBLINKS")
    for i in range(len(subfield_content)):
        subfield_content[i] = subfield_content[i].split(": ")
    # add to full-dict
    full_dict["ids"] = {x[0]:x[1] for x in subfield_content}

    # ===== FORMULA =====
    subfield_content = grab_subfield(contents, "FORMULA")
    full_dict["formula"] = grab_subfield(contents, "FORMULA")

    # ===== REACTION =====
    subfield_content = grab_subfield(contents, "REACTION")
    # expand and flatten
    subfield_content = "".join(subfield_content).split(" ") # join then split
    full_dict["reactions"] = subfield_content

    # ===== PATHWAY =====
    subfield_content = grab_subfield(contents, "PATHWAY")
    full_dict["pathways"] = [x.replace("  ", "|") for x in subfield_content]
    
    return full_dict


def get_compound_info(cpd_str, base_url="http://rest.kegg.jp/get/", flat_object=True, verbose=False):
    """Wrapper function for parse_compound_info(), to include request.
    """
    

In [2]:
url = "http://rest.kegg.jp/get/"
payload = "cpd:C01290"

r = requests.get(f"{url}{payload}")
contents = r.text.split("\n")

cpd_dict = parse_compound_info(contents)